In [ ]:
### Danish Meteorological Institute Climate Data API
# 230242300
# 18/8/24
# Python script to derive several climate datasets from DMI Climate Data API

# This script will not run, key information including variables name, temporal resolution, date range and API key have been excluded to preserve annonimity

! pip install geojson

# Libraries
import os
import json
from google.colab import drive
import requests
from shapely.geometry import shape
from datetime import datetime, timedelta
import geojson


# Mount Drive
drive.mount('/content/drive')

# Climate data URL and API key, API key has been obscured for privacy reasons
base_url = "https://dmigw.govcloud.dk/v2/climateData/collections/10kmGridValue/items"
api_key = "insert API key here" # API key removed to preserve anonymity

# generate_yearly_date_ranges is defined to get yearly date ranges (31st March - 31st October)
def generate_yearly_date_ranges(start_year, end_year):
    date_ranges = []

    for year in range(start_year, end_year + 1):
        start_date = datetime(year, 3, 31)
        end_date = datetime(year, 10, 31)


        current_date = start_date
        while current_date <= end_date:
            next_date = current_date + timedelta(days=1)
            date_range = f"{current_date.isoformat()}Z/{next_date.isoformat()}Z"
            date_label = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
            date_ranges.append((date_label, date_range))
            current_date = next_date

    return date_ranges

# Start and end years (2011 - start, 2023 - end)
start_year = 2011
end_year = 2023

# Generate date ranges from March 31st to October 31st for each year between 2011 and 2023
date_ranges = generate_yearly_date_ranges(start_year, end_year)

# create output folder to store ouput geojsons
save_path = "/content/drive/My Drive/variable_name"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Iterate through each date range and download the data
for date_label, date_range in date_ranges:
    params = {
        "datetime": date_range,
        "parameterId": "variable_name", # variable has been removed
        "timeResolution": "Insert time resolution parameter", # time resolution removed
        "bbox-crs": "https://www.opengis.net/def/crs/OGC/1.3/CRS84", # define the coordinates of the bounding box
        "api-key": api_key
    }

    # This is the get request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        print(f"Data retrieved successfully for {date_label}.")

        # Extract features and convert to GeoJSON
        features = []
        for item in data['features']:
            geometry = shape(item['geometry']) # in the case of number of frost days: geometry = Point(coordinates) was used (because it was point data)
            properties = item['properties']
            feature = geojson.Feature(geometry=geometry, properties=properties)
            features.append(feature)

        # Create feature collection for each .geojson
        feature_collection = geojson.FeatureCollection(features)

        # Define the path to save the file in Google Drive
        file_name = f"variable_name_{date_label}.geojson" # modified denpending on the input variable
        file_path = f"{save_path}/{file_name}"

        # Save the GeoJSON to Google Drive
        with open(file_path, 'w') as f:
            geojson.dump(feature_collection, f)

        print(f"Data saved to {file_path}")

    else:
        print(f"Failed to retrieve data for {date_label}. Status code: {response.status_code}")
        print(response.text)

Streaming output truncated to the last 5000 lines.
Data retrieved successfully for 2012-06-20.
Data saved to /content/drive/My Drive/max_temp_w_date/max_temp_w_date_2012-06-20.geojson
Data retrieved successfully for 2012-06-21.
Data saved to /content/drive/My Drive/max_temp_w_date/max_temp_w_date_2012-06-21.geojson
Data retrieved successfully for 2012-06-22.
Data saved to /content/drive/My Drive/max_temp_w_date/max_temp_w_date_2012-06-22.geojson
Data retrieved successfully for 2012-06-23.
Data saved to /content/drive/My Drive/max_temp_w_date/max_temp_w_date_2012-06-23.geojson
Data retrieved successfully for 2012-06-24.
Data saved to /content/drive/My Drive/max_temp_w_date/max_temp_w_date_2012-06-24.geojson
Data retrieved successfully for 2012-06-25.
Data saved to /content/drive/My Drive/max_temp_w_date/max_temp_w_date_2012-06-25.geojson
Data retrieved successfully for 2012-06-26.
Data saved to /content/drive/My Drive/max_temp_w_date/max_temp_w_date_2012-06-26.geojson
Data retrieved suc